In [2]:
#Tercero Ingredient-Based Reccomender
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

In [3]:
#Imports Tercero Data
tData = pd.read_csv('df_tercero_all.csv', low_memory = False)

tData.head(10)

,Unnamed: 0,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
0,0,French Toast,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal']",NaN,1 SV,589.38,28.98,60.73,15.86,Dairy/Egg/Gluten/Soy/Soybean Oil/Wheat,NaN
1,1,Turkey Sausage Patties,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",['Nourish'],Griddled turkey sausage patty,patty,100.80,6.30,0.00,8.82,No Major Allergens,"Turkey Sausage, Canola Oil"
2,2,Scrambled Eggs,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']",Softly scrambled cage-free eggs,3 oz,131.74,8.95,0.00,11.11,Egg,"Cage-free eggs, canola oil"
3,3,Fries Wedges Spicy,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']",Crispy-fried and seasoned potato wedges,3 oz,217.89,15.31,18.34,1.71,*Ingredient prepared in a fryer that`s not all...,"Potato wedges (potatoes, vegetable oil [soybea..."
4,4,Fiesta Scrambler,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']","Softly scrambled cage-free eggs with salsa, ch...",3.5 oz,95.10,5.18,0.78,10.15,Egg/Dairy,"Cage-free egg whites, canola oil"
5,5,Tofu Scramble,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']","Scrambled firm tofu cooked with onions, bell p...",3 oz,83.70,5.27,3.90,6.91,Soy/Gluten/Wheat,"Organic firm tofu (contains soy), yellow onion..."
6,6,Breakfast Bar,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",[],NaN,3 oz,685.82,58.42,4.00,36.51,All Major Allergens,NaN
7,7,Vanilla Chia Seed Pudding with Blueberrries,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']","Vanilla chia seed pudding with blueberries, sh...",1 serving,448.82,41.51,20.46,5.24,Tree Nuts/Coconut,"Coconut milk, chia seeds, beet sugar, imitatio..."
8,8,Housemade Granola,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']",Crunchy and sweet granola made with mixed nuts...,1/2 cup,360.18,17.33,47.13,8.03,Gluten/Wheat/Dairy/Peanuts/Tree Nuts,"Old fashioned rolled oats (per manufacturer, m..."
9,9,Fruit Salad,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']",A delicious and nutritious blend of fresh fruit!,1/2 Cup,42.30,0.64,9.79,0.49,NaN,"Cantaloupe, pineapple, red grapes, honeydew melon"


In [18]:
#Add a filter for breakfast(will be used as an example)

mtData = tData[tData['Mealtime'] == "Breakfast"]

mtData.tail(10)

,Unnamed: 0,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
380,380,House Roasted Salsa,Go Live,Breakfast,"Saturday, February 05, 2022","['Vegan', 'Halal', 'Nourish']",Spicy chili peppers and Russell Ranch tomatoes...,2 tbsp,10.15,0.40,1.60,0.32,NaN,"Green Jalapeno pepper, yellow onion, Russel Ra..."
381,381,Eggs Fried to Order,Go Live,Breakfast,"Saturday, February 05, 2022","['Vegetarian', 'Halal', 'Nourish']",Consuming raw or undercooked eggs may increase...,each,71.50,4.75,0.36,6.28,Egg,Eggs
382,382,Vanilla Chia Seed Pudding with Blueberrries,Fresh Inspirations,Breakfast,"Saturday, February 05, 2022","['Vegan', 'Halal']","Vanilla chia seed pudding with blueberries, sh...",1 SV,448.79,41.51,20.46,5.24,Tree Nuts/Coconut,"Coconut milk, chia seeds, beet sugar, imitatio..."
383,383,Housemade Granola,Fresh Inspirations,Breakfast,"Saturday, February 05, 2022","['Vegetarian', 'Halal', 'Nourish']",Crunchy and sweet granola made with mixed nuts...,1/2 cup,360.18,17.33,47.13,8.03,Gluten/Wheat/Dairy/Peanuts/Tree Nuts,"Old fashioned rolled oats (per manufacturer, m..."
384,384,Fruit Salad,Fresh Inspirations,Breakfast,"Saturday, February 05, 2022","['Vegan', 'Halal', 'Nourish']",A delicious and nutritious blend of fresh fruit!,1/2 Cup,42.30,0.64,9.79,0.49,NaN,"Cantaloupe, pineapple, red grapes, honeydew melon"
385,385,Oatmeal,Fresh Inspirations,Breakfast,"Saturday, February 05, 2022","['Vegan', 'Halal', 'Nourish']",Hot old-fashioned rolled oats (may contain whe...,7 oz,105.04,1.94,18.85,3.60,Gluten/Wheat,"Whole grain oats (per manufacturer, may contai..."
386,386,Hard Boiled Eggs,Fresh Inspirations,Breakfast,"Saturday, February 05, 2022","['Vegetarian', 'Halal', 'Nourish']","Cage-free, hard boiled eggs",1 each,71.50,4.76,0.36,6.28,Egg,Cage free eggs
387,387,Mini Plain Bagel,Harvest Bakery,Breakfast,"Saturday, February 05, 2022","['Vegan', 'Halal']","Per manufacturer, produced in a facility that ...",1 bagel,359.99,1.50,71.00,12.99,Gluten/Soybean Oil/Wheat,"Enriched unbleached flour (wheat flour, malted..."
388,388,Zucchini Bread,Harvest Bakery,Breakfast,"Saturday, February 05, 2022",['Vegetarian'],"(may contain: peanuts, tree nuts)",1 slice,95.21,4.39,13.18,1.10,Egg/Gluten/Soybean Oil/Wheat,"Unbleached Wheat Flour (Wheat Flour, Malted Ba..."
389,389,Raspberry Peach Danish,Harvest Bakery,Breakfast,"Saturday, February 05, 2022",['Vegetarian'],"(may contain: peanuts, tree nuts)",each,180.00,10.00,18.00,3.00,Dairy/Egg/Gluten/Wheat,"Unbleached Enriched Flour (Wheat Flour, Malted..."


In [6]:
#Filter it even more by the specific date for the meals
ftData = mtData[mtData['Date'] == "Sunday, January 30, 2022"]
ftData.tail(10)

,Unnamed: 0,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
6,6,Breakfast Bar,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",[],NaN,3 oz,685.82,58.42,4.00,36.51,All Major Allergens,NaN
7,7,Vanilla Chia Seed Pudding with Blueberrries,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']","Vanilla chia seed pudding with blueberries, sh...",1 serving,448.82,41.51,20.46,5.24,Tree Nuts/Coconut,"Coconut milk, chia seeds, beet sugar, imitatio..."
8,8,Housemade Granola,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']",Crunchy and sweet granola made with mixed nuts...,1/2 cup,360.18,17.33,47.13,8.03,Gluten/Wheat/Dairy/Peanuts/Tree Nuts,"Old fashioned rolled oats (per manufacturer, m..."
9,9,Fruit Salad,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']",A delicious and nutritious blend of fresh fruit!,1/2 Cup,42.30,0.64,9.79,0.49,NaN,"Cantaloupe, pineapple, red grapes, honeydew melon"
10,10,Oatmeal,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']",Hot old-fashioned rolled oats (may contain whe...,7 oz,105.04,1.94,18.85,3.60,Gluten/Wheat,"Whole grain oats (per manufacturer, may contai..."
11,11,Hard Boiled Eggs,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']","Cage-free, hard boiled eggs",1 each,71.50,4.76,0.36,6.28,Egg,Cage free eggs
12,12,Mini Plain Bagel,Harvest Bakery,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']","Per manufacturer, produced in a facility that ...",1 bagel,359.99,1.50,71.00,12.99,Gluten/Soybean Oil/Wheat,"Enriched unbleached flour (wheat flour, malted..."
13,13,Double Chocolate Muffin,Harvest Bakery,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal']","Produced in a facility that also uses dairy, e...",each,440.00,25.00,53.00,5.00,Dairy/Egg/Gluten/Soy/Soybean Oil/Wheat,"Sugar, enriched bleached wheat flour, cocoa, m..."
14,14,Cream Cheese Frosted Cinnamon Rolls,Harvest Bakery,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal']","Produced in a facility that also uses milk, eg...",each,230.00,12.00,25.00,4.00,Dairy/Gluten/Wheat,"Unbleached enriched flour (wheat flour, malted..."
15,15,Crumb Donut,Harvest Bakery,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']","Produced in a facility that also uses dairy, e...",each,150.00,7.00,21.00,2.00,Gluten/Soy/Soybean Oil/Wheat,Donut: unbleached enriched wheat flour (wheat ...


In [7]:
#This version takes a food, and reccommends similar items based off its contents of ingredients 
#BASED ON MEALTIME
mtData['Ingredients'].head()

0                                                  NaN
1                           Turkey Sausage, Canola Oil
2                           Cage-free eggs, canola oil
3    Potato wedges (potatoes, vegetable oil [soybea...
4                     Cage-free egg whites, canola oil
Name: Ingredients, dtype: object

In [8]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
mtData['Ingredients'] = mtData['Ingredients'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(mtData['Ingredients'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

C:\Users\19255\AppData\Local\Temp\ipykernel_4076\1620882652.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mtData['Ingredients'] = mtData['Ingredients'].fillna('')


(135, 395)

In [14]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and food items
indices = pd.Series(mtData.index, index=mtData['Name']).drop_duplicates()
indices

Name
French Toast                0
Turkey Sausage Patties      1
Scrambled Eggs              2
Fries Wedges Spicy          3
Fiesta Scrambler            4
                         ... 
Oatmeal                   385
Hard Boiled Eggs          386
Mini Plain Bagel          387
Zucchini Bread            388
Raspberry Peach Danish    389
Length: 135, dtype: int64

In [16]:
# Function that takes in food item as input and outputs most similar foods in terms of ingredients
def get_recommendations(food, cosine_sim=cosine_sim):
    # Get the index of the food that matches the ingredient
    idx = indices[food]

    # Get the pairwsie similarity scores of all food item with that food item
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the food items based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar food items
    sim_scores = sim_scores[1:11]

    # Get the food item indices
    food_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar food items
    return mtData.iloc[food_indices]


In [17]:
get_recommendations("French Toast")

,Unnamed: 0,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
1,1,Turkey Sausage Patties,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",['Nourish'],Griddled turkey sausage patty,patty,100.80,6.30,0.00,8.82,No Major Allergens,"Turkey Sausage, Canola Oil"
2,2,Scrambled Eggs,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']",Softly scrambled cage-free eggs,3 oz,131.74,8.95,0.00,11.11,Egg,"Cage-free eggs, canola oil"
3,3,Fries Wedges Spicy,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']",Crispy-fried and seasoned potato wedges,3 oz,217.89,15.31,18.34,1.71,*Ingredient prepared in a fryer that`s not all...,"Potato wedges (potatoes, vegetable oil [soybea..."
4,4,Fiesta Scrambler,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']","Softly scrambled cage-free eggs with salsa, ch...",3.5 oz,95.10,5.18,0.78,10.15,Egg/Dairy,"Cage-free egg whites, canola oil"
5,5,Tofu Scramble,Pacific Fusion,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']","Scrambled firm tofu cooked with onions, bell p...",3 oz,83.70,5.27,3.90,6.91,Soy/Gluten/Wheat,"Organic firm tofu (contains soy), yellow onion..."
6,6,Breakfast Bar,Pacific Fusion,Breakfast,"Sunday, January 30, 2022",[],NaN,3 oz,685.82,58.42,4.00,36.51,All Major Allergens,
7,7,Vanilla Chia Seed Pudding with Blueberrries,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal']","Vanilla chia seed pudding with blueberries, sh...",1 serving,448.82,41.51,20.46,5.24,Tree Nuts/Coconut,"Coconut milk, chia seeds, beet sugar, imitatio..."
8,8,Housemade Granola,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegetarian', 'Halal', 'Nourish']",Crunchy and sweet granola made with mixed nuts...,1/2 cup,360.18,17.33,47.13,8.03,Gluten/Wheat/Dairy/Peanuts/Tree Nuts,"Old fashioned rolled oats (per manufacturer, m..."
9,9,Fruit Salad,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']",A delicious and nutritious blend of fresh fruit!,1/2 Cup,42.30,0.64,9.79,0.49,NaN,"Cantaloupe, pineapple, red grapes, honeydew melon"
10,10,Oatmeal,Fresh Inspirations,Breakfast,"Sunday, January 30, 2022","['Vegan', 'Halal', 'Nourish']",Hot old-fashioned rolled oats (may contain whe...,7 oz,105.04,1.94,18.85,3.60,Gluten/Wheat,"Whole grain oats (per manufacturer, may contai..."
